In [ ]:
#Inspirasjon / kilder = https://stackabuse.com/linear-regression-in-python-with-scikit-learn/
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv('/Users/eliasnilsen/Desktop/Studie/VÅR2021/INFO284/Exam/nyc-rolling-sales.csv')

In [ ]:
data.head()

In [ ]:
#Sletter ubrukelig data
data.drop('Unnamed: 0', axis=1, inplace=True)
data.drop('EASE-MENT', axis=1, inplace=True)

In [ ]:
#Sjekker for duplicates i datasettet
sum(data.duplicated())

In [ ]:
#Sletter duplicates og sjekker om det funket
data = data.drop_duplicates(data.columns, keep='last')
sum(data.duplicated())

In [ ]:
#Noen av feltene inneholder et - tegn, vi bytter ut disse med 0
#' -  ' finnes i en eller flere celler, vi må fjerne denne for å unngå errors
data['GROSS SQUARE FEET'] = data['GROSS SQUARE FEET'].str.strip()
data['SALE PRICE'] = data['SALE PRICE'].str.strip()
data.replace({"-":0},inplace=True)
data.replace(' ',np.nan, inplace=True)
#Tomt felt som ikke er satt opp riktig
data.replace(' -  ', np.nan)
df1=data.copy()

In [ ]:
#Sjekker datatypen i settet
data.info()

In [ ]:
df1.dropna()
df1[['GROSS SQUARE FEET', 'SALE PRICE']] = df1[['GROSS SQUARE FEET', 'SALE PRICE']].astype(int)
df1.info()
df1.head()

In [ ]:
#Vi vil bruke dataen fra GROSS SQUARE FEET kollonen til å forutse SALE PRICE
X = df1['GROSS SQUARE FEET'].values.reshape(-1, 1)
y = df1['SALE PRICE'].values.reshape(-1, 1)

In [ ]:
#Deler opp dataen til trening og testing av modellen
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
#Trener dataen
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

In [ ]:
#Intercept og slope
print(X.shape)
print(y.shape)
print("regressor.coef_: {}".format(regressor.coef_))
print("regressor.intercept_: {}".format(regressor.intercept_))

In [ ]:
print("Training set score:  {:.2f}".format(regressor.score(X_train,y_train)))
print("Test set score:  {:.2f}".format(regressor.score(X_train,y_test)))